In [1]:
import pandas as pd
import pickle

In [2]:
df = pd.read_csv('phil_nlp.csv')

df.sample(5)

,title,author,school,sentence_spacy,sentence_str,original_publication_date,corpus_edition_date,sentence_length,sentence_lowered,tokenized_txt,lemmatized_str
320174,On The Principles Of Political Economy And Tax...,Ricardo,capitalism,The operation of the system of poor laws has b...,The operation of the system of poor laws has b...,1817,2010,76,the operation of the system of poor laws has b...,"['the', 'operation', 'of', 'the', 'system', 'o...",the operation of the system of poor law have ...
197737,The Order Of Things,Foucault,continental,"Neither memory nor imagination, nor, therefore...","Neither memory nor imagination, nor, therefore...",1966,2002,77,"neither memory nor imagination, nor, therefore...","['neither', 'memory', 'nor', 'imagination', 'n...","neither memory nor imagination , nor , theref..."
350388,Proslogion,Anselm,scholasticism,"†'The wife is not exactly a partner, but it is...","†'The wife is not exactly a partner, but it is...",1077,2000,77,"†'the wife is not exactly a partner, but it is...","['the', 'wife', 'is', 'not', 'exactly', 'partn...","†'the wife be not exactly a partner , but -PR..."
156280,Lewis - Papers,Lewis,analytic,As of course they are.,As of course they are.,1985,2008,22,as of course they are.,"['as', 'of', 'course', 'they', 'are']",as of course -PRON- be .
205805,Writing And Difference,Derrida,continental,"Through a ruse of life, that is, of reason, li...","Through a ruse of life, that is, of reason, li...",1967,2001,71,"through a ruse of life, that is, of reason, li...","['through', 'ruse', 'of', 'life', 'that', 'is'...","through a ruse of life , that is , of reason ..."


In [3]:
len(df)

371354

In [4]:
for_db = df
for_db['date'] = for_db['original_publication_date']
for_db['date'] = for_db['date'].apply(lambda x: str(x)[1:]+' BC' if x < 0 else str(x))
for_db['sentence'] = for_db['sentence_str']
for_db['school'] = for_db['school'].apply(lambda x: x.replace('_', ' ').title())
for_db = for_db.drop(['sentence_spacy', 
                      'sentence_length',
                      'sentence_lowered', 
                      'sentence_str', 
                      'tokenized_txt', 
                      'lemmatized_str',
                      'corpus_edition_date',
                      'original_publication_date'], axis=1)
for_db.columns = [i.upper() for i in for_db.columns]

for_db.sample(5)


,TITLE,AUTHOR,SCHOOL,DATE,SENTENCE
73012,Aristotle - Complete Works,Aristotle,Aristotle,320 BC,And similarly with regard to lifeless things; ...
287527,Elements Of The Philosophy Of Right,Hegel,German Idealism,1820,"For it is truly present in God alone, and when..."
296549,Capital,Marx,Communism,1883,"This is a superior class of workmen, some of t..."
81826,Aristotle - Complete Works,Aristotle,Aristotle,320 BC,Others again grew out of the class which were ...
61987,Aristotle - Complete Works,Aristotle,Aristotle,320 BC,As to the animals that are not polydactylous b...


In [5]:
for_db.sample(5)

,TITLE,AUTHOR,SCHOOL,DATE,SENTENCE
262833,The System Of Ethics,Fichte,German Idealism,1798,"More specifically, how am I supposed to recogn..."
36429,Plato - Complete Works,Plato,Plato,350 BC,"What will happen if we do not, I will not say,..."
101861,A Treatise Of Human Nature,Hume,Empiricism,1739,"Though possession be stable, men may often rea..."
13557,Plato - Complete Works,Plato,Plato,350 BC,Consider it this way.
66239,Aristotle - Complete Works,Aristotle,Aristotle,320 BC,"The reason of the increased vibration is that,..."


In [6]:
for_db['SCHOOL'].value_counts(normalize=True)

Analytic           0.149251
Aristotle          0.131354
German Idealism    0.113455
Plato              0.103314
Continental        0.090962
Phenomenology      0.076943
Rationalism        0.061798
Empiricism         0.053671
Scholasticism      0.050184
Capitalism         0.048994
Communism          0.048358
Feminism           0.028275
Stoicism           0.020342
Nietzsche          0.018349
Kierkegaard        0.004750
Name: SCHOOL, dtype: float64

In [7]:
classifier_dict = {}
for author in for_db['AUTHOR'].unique():
  classifier_dict[author] = 'AUTHOR'
for title in for_db['TITLE'].unique():
  classifier_dict[title] = 'TITLE'
for school in for_db['SCHOOL'].unique():
  classifier_dict[school] = 'SCHOOL'

In [8]:
classifier_dict

{'Plato': 'SCHOOL',
 'Aristotle': 'SCHOOL',
 'Locke': 'AUTHOR',
 'Hume': 'AUTHOR',
 'Berkeley': 'AUTHOR',
 'Spinoza': 'AUTHOR',
 'Leibniz': 'AUTHOR',
 'Descartes': 'AUTHOR',
 'Malebranche': 'AUTHOR',
 'Russell': 'AUTHOR',
 'Moore': 'AUTHOR',
 'Wittgenstein': 'AUTHOR',
 'Lewis': 'AUTHOR',
 'Quine': 'AUTHOR',
 'Popper': 'AUTHOR',
 'Kripke': 'AUTHOR',
 'Foucault': 'AUTHOR',
 'Derrida': 'AUTHOR',
 'Deleuze': 'AUTHOR',
 'Merleau-Ponty': 'AUTHOR',
 'Husserl': 'AUTHOR',
 'Heidegger': 'AUTHOR',
 'Kant': 'AUTHOR',
 'Fichte': 'AUTHOR',
 'Hegel': 'AUTHOR',
 'Marx': 'AUTHOR',
 'Lenin': 'AUTHOR',
 'Smith': 'AUTHOR',
 'Ricardo': 'AUTHOR',
 'Keynes': 'AUTHOR',
 'Epictetus': 'AUTHOR',
 'Marcus Aurelius': 'AUTHOR',
 'Nietzsche': 'SCHOOL',
 'Wollstonecraft': 'AUTHOR',
 'Beauvoir': 'AUTHOR',
 'Davis': 'AUTHOR',
 'Augustine': 'AUTHOR',
 'Anselm': 'AUTHOR',
 'Seneca': 'AUTHOR',
 'Kierkegaard': 'SCHOOL',
 'Plato - Complete Works': 'TITLE',
 'Aristotle - Complete Works': 'TITLE',
 'Second Treatise On Governm

Now get an updated dropdown menu.

In [9]:
all_options = sorted([x for x in list(classifier_dict.keys())])

In [10]:
dropdown_menu = []
for source in all_options:
    dropdown_menu.append({'label': source, 'value': source})

dropdown_menu

[{'label': 'A General Theory Of Employment, Interest, And Money',
  'value': 'A General Theory Of Employment, Interest, And Money'},
 {'label': 'A Treatise Concerning The Principles Of Human Knowledge',
  'value': 'A Treatise Concerning The Principles Of Human Knowledge'},
 {'label': 'A Treatise Of Human Nature',
  'value': 'A Treatise Of Human Nature'},
 {'label': 'Analytic', 'value': 'Analytic'},
 {'label': 'Anselm', 'value': 'Anselm'},
 {'label': 'Anti-Oedipus', 'value': 'Anti-Oedipus'},
 {'label': 'Aristotle', 'value': 'Aristotle'},
 {'label': 'Aristotle - Complete Works',
  'value': 'Aristotle - Complete Works'},
 {'label': 'Augustine', 'value': 'Augustine'},
 {'label': 'Beauvoir', 'value': 'Beauvoir'},
 {'label': 'Being And Time', 'value': 'Being And Time'},
 {'label': 'Berkeley', 'value': 'Berkeley'},
 {'label': 'Beyond Good And Evil', 'value': 'Beyond Good And Evil'},
 {'label': 'Capital', 'value': 'Capital'},
 {'label': 'Capitalism', 'value': 'Capitalism'},
 {'label': 'Communi

The next cell makes pickles of each relevant slice. Not necessary since we run it all on SQL.

In [ ]:
for source in classifier_dict.keys():
        source_slice = df[df[classifier_dict[source]]==source]
        source = source.title()
        source_pkl = open(f'../search_app/slice_pickles/{source}_slice.pkl', 'wb')
        pickle.dump(source_slice, source_pkl)
        source_pkl.close()
